In [2]:
import os
import time
import requests
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

# URL for the pre-trained fastText German model
fasttext_url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz'
model_filename = 'cc.de.300.vec.gz'

# Function to download the model if it's not already present
def download_model(url, filename):
    if not os.path.exists(filename):
        print(f'Downloading {filename}...')
        response = requests.get(url, stream=True)
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print(f'{filename} downloaded.')
    else:
        print(f'{filename} already exists.')

# Download the fastText German model if it doesn't already exist
download_model(fasttext_url, model_filename)

# Load the pre-trained fastText German model (binary=False because it's in text format)
print("Loading the Word2Vec model...")
word2vec_model = KeyedVectors.load_word2vec_format(model_filename, binary=False)
print("Model loaded successfully.")

# Function to calculate average Word2Vec vector for a sentence
def sentence_vector(sentence, model, vector_size):
    words = sentence.split()
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)  # Return a zero vector if no words are found in the model
    return np.mean(word_vectors, axis=0)

# Define the Word2Vec similarity function
def word2vec_similarity(text1, text2, model, vector_size):
    vec1 = sentence_vector(text1, model, vector_size)
    vec2 = sentence_vector(text2, model, vector_size)
    
    # Check if either vector is all zeros (i.e., no valid words)
    if np.all(vec1 == 0) or np.all(vec2 == 0):
        return 0  # Assign 0 similarity if one of the sentences is empty or has no valid words
    
    # Access the correct element of the similarity matrix
    return cosine_similarity([vec1], [vec2])[0, 0]

# Load the CSV file with test sentences
df = pd.read_csv('testsätze.csv')

# Prepare results
results = []

# Iterate over each row to compare the sentences
for index, row in df.iterrows():
    # Extract the sentences from each column
    satz1 = row.get('Satz 1', "")
    satz2 = row.get('Satz 2', "")
    satz3 = row.get('Satz 3', "")

    # List of comparisons: Satz 1 with Satz 2, Satz 1 with Satz 3, Satz 2 with Satz 3
    comparisons = [("Satz 1 mit Satz 2", satz1, satz2), 
                   ("Satz 1 mit Satz 3", satz1, satz3),
                   ("Satz 2 mit Satz 3", satz2, satz3)]
    
    # Compare each sentence pair and calculate the Word2Vec cosine similarity
    for comparison_label, text1, text2 in comparisons:
        if not text1 or not text2:
            continue
        
        start_time = time.perf_counter()
        similarity = word2vec_similarity(text1, text2, word2vec_model, vector_size=300)  # fastText uses 300 dimensions
        end_time = time.perf_counter()
        
        elapsed_time_ms = round((end_time - start_time) * 1000, 9)
        
        results.append({
            "Vergleich": comparison_label,
            "Word2Vec Ähnlichkeit (Cosine)": similarity,
            "Berechnungszeit (ms)": elapsed_time_ms
        })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Print the results in a formatted DataFrame
print(results_df.to_string(index=False))


cc.de.300.vec.gz already exists.
Loading the Word2Vec model...
Model loaded successfully.
        Vergleich  Word2Vec Ähnlichkeit (Cosine)  Berechnungszeit (ms)
Satz 1 mit Satz 2                       0.989480                0.4458
Satz 1 mit Satz 3                       0.920521                0.2510
Satz 2 mit Satz 3                       0.918425                0.2223


In [3]:
import os
import time
import pandas as pd
from angle_emb import AnglE  # Prompts importieren wir nicht, da es zu einem Fehler führt
from sklearn.metrics.pairwise import cosine_similarity

# Funktion zum Initialisieren des AnglE-Modells
def load_angle_model(base_model_name, pretrained_lora_path):
    print(f"Lade Modell {pretrained_lora_path} mit Basis-Modell {base_model_name}...")
    angle = AnglE.from_pretrained(base_model_name, pretrained_lora_path=pretrained_lora_path)
    print(f"Modell erfolgreich geladen.")
    return angle

# Funktion zum Enkodieren von Sätzen und Berechnung der Ähnlichkeit
def angle_similarity(text1, text2, model):
    # Sätze enkodieren
    vec1 = model.encode({'text': text1}, to_numpy=True)
    vec2 = model.encode({'text': text2}, to_numpy=True)
    
    # Cosine Similarity berechnen
    similarity = cosine_similarity([vec1], [vec2])[0, 0]
    
    return similarity

# Das Modell von AnglE und den LoRA-Pfad laden
base_model_name = "NousResearch/Llama-2-7b-hf"
pretrained_lora_path = "SeanLee97/angle-llama-7b-nli-v2"
model = load_angle_model(base_model_name, pretrained_lora_path)

# Lade die CSV-Datei mit den Testsätzen
df = pd.read_csv('testsätze.csv')

# Ergebnisse vorbereiten
results = []

# Iteriere über jede Zeile, um die Sätze zu vergleichen
for index, row in df.iterrows():
    # Extrahiere die Sätze aus jeder Spalte
    satz1 = row.get('Satz 1', "")
    satz2 = row.get('Satz 2', "")
    satz3 = row.get('Satz 3', "")

    # Liste der Vergleiche: Satz 1 mit Satz 2, Satz 1 mit Satz 3, Satz 2 mit Satz 3
    comparisons = [("Satz 1 mit Satz 2", satz1, satz2), 
                   ("Satz 1 mit Satz 3", satz1, satz3),
                   ("Satz 2 mit Satz 3", satz2, satz3)]
    
    # Vergleiche jedes Satzpaar und berechne die Cosine Similarity
    for comparison_label, text1, text2 in comparisons:
        if not text1 or not text2:
            continue
        
        start_time = time.perf_counter()
        similarity = angle_similarity(text1, text2, model)
        end_time = time.perf_counter()
        
        elapsed_time_ms = round((end_time - start_time) * 1000, 9)
        
        # Ergebnisse speichern
        results.append({
            "Vergleich": comparison_label,
            "Cosine Similarity": similarity,
            "Berechnungszeit (ms)": elapsed_time_ms
        })

# Erstelle ein DataFrame aus den Ergebnissen
results_df = pd.DataFrame(results)

# Gib die Ergebnisse in einem formatierten DataFrame aus
print(results_df.to_string(index=False))


INFO:AnglE:LLM detected, automatically set is_llm=True.If it is wrong, you can manually set `is_llm`.
INFO:AnglE:🚨 LLM detected, but pooling strategy is specified to cls.Please check whether it is correct. It is recommended to use `last` pooling strategy for LLM.
INFO:AnglE:LLM detected, automatically set apply_lora=True.If it is wrong, you can manually set `apply_lora`.


Lade Modell SeanLee97/angle-llama-7b-nli-v2 mit Basis-Modell NousResearch/Llama-2-7b-hf...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:AnglE:Load lora weight from SeanLee97/angle-llama-7b-nli-v2


Modell erfolgreich geladen.


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [6]:
import os
import time
import pandas as pd
from angle_emb import AnglE
from sklearn.metrics.pairwise import cosine_similarity

# Funktion zum Initialisieren des AnglE-Modells
def load_angle_model(base_model_name, pretrained_lora_path):
    print(f"Lade Modell {pretrained_lora_path} mit Basis-Modell {base_model_name}...")
    angle = AnglE.from_pretrained(base_model_name, pretrained_lora_path=pretrained_lora_path)
    print(f"Modell erfolgreich geladen.")
    return angle

# Funktion zum Enkodieren von Sätzen und Berechnung der Ähnlichkeit
def angle_similarity(text1, text2, model):
    # Überprüfen, ob die Eingaben Strings sind
    if not isinstance(text1, str) or not isinstance(text2, str):
        raise ValueError("Beide Texteingaben müssen Strings sein.")
    
    # Sätze enkodieren
    vec1 = model.encode(text1, to_numpy=True)  # Direkter String als Input
    vec2 = model.encode(text2, to_numpy=True)  # Direkter String als Input
    
    # Debugging: Die Dimensionen der Vektoren ausgeben
    print(f"Shape of vec1: {vec1.shape}")
    print(f"Shape of vec2: {vec2.shape}")
    
    # Überprüfen, ob die Vektoren 1D oder mehrdimensional sind
    if len(vec1.shape) > 2 or len(vec2.shape) > 2:
        raise ValueError(f"Found array with dim {vec1.shape}, but expected <= 2.")
    
    # Wenn sie 1D sind, in 2D umformen
    if len(vec1.shape) == 1:
        vec1 = vec1.reshape(1, -1)
    if len(vec2.shape) == 1:
        vec2 = vec2.reshape(1, -1)
    
    # Cosine Similarity berechnen
    similarity = cosine_similarity(vec1, vec2)[0, 0]
    
    return similarity


# Das Modell von AnglE und den LoRA-Pfad laden
base_model_name = "NousResearch/Llama-2-7b-hf"
pretrained_lora_path = "SeanLee97/angle-llama-7b-nli-v2"
model = load_angle_model(base_model_name, pretrained_lora_path)

# Lade die CSV-Datei mit den Testsätzen
df = pd.read_csv('testsätze.csv')

# Ergebnisse vorbereiten
results = []

# Iteriere über jede Zeile, um die Sätze zu vergleichen
for index, row in df.iterrows():
    # Extrahiere die Sätze aus jeder Spalte
    satz1 = row.get('Satz 1', "")
    satz2 = row.get('Satz 2', "")
    satz3 = row.get('Satz 3', "")

    # Liste der Vergleiche: Satz 1 mit Satz 2, Satz 1 mit Satz 3, Satz 2 mit Satz 3
    comparisons = [("Satz 1 mit Satz 2", satz1, satz2), 
                   ("Satz 1 mit Satz 3", satz1, satz3),
                   ("Satz 2 mit Satz 3", satz2, satz3)]
    
    # Vergleiche jedes Satzpaar und berechne die Cosine Similarity
    for comparison_label, text1, text2 in comparisons:
        if not text1 or not text2:
            continue
        
        start_time = time.perf_counter()
        similarity = angle_similarity(text1, text2, model)
        end_time = time.perf_counter()
        
        elapsed_time_ms = round((end_time - start_time) * 1000, 9)
        
        # Ergebnisse speichern
        results.append({
            "Vergleich": comparison_label,
            "Cosine Similarity": similarity,
            "Berechnungszeit (ms)": elapsed_time_ms
        })

# Erstelle ein DataFrame aus den Ergebnissen
results_df = pd.DataFrame(results)

# Gib die Ergebnisse in einem formatierten DataFrame aus
print(results_df.to_string(index=False))

INFO:AnglE:LLM detected, automatically set is_llm=True.If it is wrong, you can manually set `is_llm`.
INFO:AnglE:🚨 LLM detected, but pooling strategy is specified to cls.Please check whether it is correct. It is recommended to use `last` pooling strategy for LLM.
INFO:AnglE:LLM detected, automatically set apply_lora=True.If it is wrong, you can manually set `apply_lora`.


Lade Modell SeanLee97/angle-llama-7b-nli-v2 mit Basis-Modell NousResearch/Llama-2-7b-hf...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:AnglE:Load lora weight from SeanLee97/angle-llama-7b-nli-v2
C:\Users\Finn\Work\Bachelor\Bachelor\venv\Lib\site-packages\peft\utils\save_and_load.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related t

KeyError: 'base_model.model.model.model.embed_tokens'

INFO:AnglE:LLM detected, automatically set is_llm=True.If it is wrong, you can manually set `is_llm`.
INFO:AnglE:🚨 LLM detected, but pooling strategy is specified to cls.Please check whether it is correct. It is recommended to use `last` pooling strategy for LLM.
INFO:AnglE:LLM detected, automatically set apply_lora=True.If it is wrong, you can manually set `apply_lora`.


Lade Modell SeanLee97/angle-llama-7b-nli-v2 mit Basis-Modell NousResearch/Llama-2-7b-hf...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:AnglE:Load lora weight from SeanLee97/angle-llama-7b-nli-v2
C:\Users\Finn\Work\Bachelor\Bachelor\venv\Lib\site-packages\peft\utils\save_and_load.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related t

KeyError: 'base_model.model.model.model.embed_tokens'

In [7]:
import os
import time
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Lade das SBERT-Modell (deutsches Modell)
print("Lade das SBERT-Modell...")
sbert_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
print("Modell erfolgreich geladen.")

# Funktion zur Berechnung der SBERT-Satz-Ähnlichkeit
def sbert_similarity(text1, text2, model):
    # Berechnung der Satz-Embeddings mit SBERT
    embeddings = model.encode([text1, text2])
    
    # Berechnung der Cosine Similarity zwischen den beiden Embeddings
    return cosine_similarity([embeddings[0]], [embeddings[1]])[0, 0]

# CSV-Datei mit Testsätzen laden
df = pd.read_csv('testsätze.csv')

# Ergebnisse vorbereiten
results = []

# Iteriere über jede Zeile, um die Sätze zu vergleichen
for index, row in df.iterrows():
    # Extrahiere die Sätze aus jeder Spalte
    satz1 = row.get('Satz 1', "")
    satz2 = row.get('Satz 2', "")
    satz3 = row.get('Satz 3', "")

    # Liste der Vergleiche: Satz 1 mit Satz 2, Satz 1 mit Satz 3, Satz 2 mit Satz 3
    comparisons = [("Satz 1 mit Satz 2", satz1, satz2), 
                   ("Satz 1 mit Satz 3", satz1, satz3),
                   ("Satz 2 mit Satz 3", satz2, satz3)]
    
    # Vergleiche jedes Satzpaar und berechne die Cosine Similarity
    for comparison_label, text1, text2 in comparisons:
        if not text1 or not text2:
            continue
        
        start_time = time.perf_counter()
        similarity = sbert_similarity(text1, text2, sbert_model)
        end_time = time.perf_counter()
        
        elapsed_time_ms = round((end_time - start_time) * 1000, 9)
        
        results.append({
            "Vergleich": comparison_label,
            "SBERT Ähnlichkeit (Cosine)": similarity,
            "Berechnungszeit (ms)": elapsed_time_ms
        })

# Erstelle ein DataFrame aus den Ergebnissen
results_df = pd.DataFrame(results)

# Ergebnisse in einem formatierten DataFrame ausgeben
print(results_df.to_string(index=False))

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: distiluse-base-multilingual-cased-v1


Lade das SBERT-Modell...


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

C:\Users\Finn\Work\Bachelor\Bachelor\venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Finn\.cache\huggingface\hub\models--sentence-transformers--distiluse-base-multilingual-cased-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Modell erfolgreich geladen.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

        Vergleich  SBERT Ähnlichkeit (Cosine)  Berechnungszeit (ms)
Satz 1 mit Satz 2                    0.909018               58.5371
Satz 1 mit Satz 3                    0.919996               23.5237
Satz 2 mit Satz 3                    0.824267               24.9107


In [8]:
import os
import time
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Lade das SBERT-Modell (deutsches Modell)
print("Lade das SBERT-Modell...")
sbert_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
print("Modell erfolgreich geladen.")

# Funktion zur Berechnung der SBERT-Satz-Ähnlichkeit
def sbert_similarity(text1, text2, model):
    # Berechnung der Satz-Embeddings mit SBERT
    embeddings = model.encode([text1, text2])
    
    # Berechnung der Cosine Similarity zwischen den beiden Embeddings
    return cosine_similarity([embeddings[0]], [embeddings[1]])[0, 0]

# CSV-Datei mit Testsätzen laden
df = pd.read_csv('testsätze.csv')

# Ergebnisse vorbereiten
results = []

# Iteriere über jede Zeile, um die Sätze zu vergleichen
for index, row in df.iterrows():
    # Extrahiere die Sätze aus jeder Spalte
    satz1 = row.get('Satz 1', "")
    satz2 = row.get('Satz 2', "")
    satz3 = row.get('Satz 3', "")

    # Liste der Vergleiche: Satz 1 mit Satz 2, Satz 1 mit Satz 3, Satz 2 mit Satz 3
    comparisons = [("Satz 1 mit Satz 2", satz1, satz2), 
                   ("Satz 1 mit Satz 3", satz1, satz3),
                   ("Satz 2 mit Satz 3", satz2, satz3)]
    
    # Vergleiche jedes Satzpaar und berechne die Cosine Similarity
    for comparison_label, text1, text2 in comparisons:
        if not text1 or not text2:
            continue
        
        start_time = time.perf_counter()
        similarity = sbert_similarity(text1, text2, sbert_model)
        end_time = time.perf_counter()
        
        elapsed_time_ms = round((end_time - start_time) * 1000, 9)
        
        results.append({
            "Vergleich": comparison_label,
            "SBERT Ähnlichkeit (Cosine)": similarity,
            "Berechnungszeit (ms)": elapsed_time_ms
        })

# Erstelle ein DataFrame aus den Ergebnissen
results_df = pd.DataFrame(results)

# Ergebnisse in einem formatierten DataFrame ausgeben
print(results_df.to_string(index=False))

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Lade das SBERT-Modell...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Finn\Work\Bachelor\Bachelor\venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Finn\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Modell erfolgreich geladen.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

        Vergleich  SBERT Ähnlichkeit (Cosine)  Berechnungszeit (ms)
Satz 1 mit Satz 2                    0.950575               89.8289
Satz 1 mit Satz 3                    0.870597               49.3794
Satz 2 mit Satz 3                    0.811649               47.4749


In [10]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load the LoRA weights
model = PeftModel.from_pretrained(base_model, pretrained_lora_path)

print("Model with LoRA loaded successfully.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\Finn\Work\Bachelor\Bachelor\venv\Lib\site-packages\peft\utils\save_and_load.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(file

Model with LoRA loaded successfully.


In [11]:
import os
import time
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Funktion zum Initialisieren des scaling_sentemb-Modells
def load_scaling_sentemb_model():
    print("Lade das scaling_sentemb Modell...")
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
    model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
    print("Modell erfolgreich geladen.")
    return tokenizer, model

# Funktion zum Enkodieren von Sätzen und Berechnung der Ähnlichkeit
def scaling_sentemb_similarity(text1, text2, tokenizer, model):
    # Tokenisierung der Sätze
    inputs_1 = tokenizer(text1, return_tensors="pt", truncation=True, padding=True)
    inputs_2 = tokenizer(text2, return_tensors="pt", truncation=True, padding=True)
    
    # Vektoren berechnen
    with torch.no_grad():
        vec1 = model(**inputs_1).last_hidden_state.mean(dim=1).numpy()
        vec2 = model(**inputs_2).last_hidden_state.mean(dim=1).numpy()
    
    # Cosine Similarity berechnen
    similarity = cosine_similarity([vec1], [vec2])[0, 0]
    
    return similarity

# Das scaling_sentemb Modell laden
tokenizer, model = load_scaling_sentemb_model()

# Lade die CSV-Datei mit den Testsätzen
df = pd.read_csv('testsätze.csv')

# Ergebnisse vorbereiten
results = []

# Iteriere über jede Zeile, um die Sätze zu vergleichen
for index, row in df.iterrows():
    # Extrahiere die Sätze aus jeder Spalte
    satz1 = row.get('Satz 1', "")
    satz2 = row.get('Satz 2', "")
    satz3 = row.get('Satz 3', "")

    # Liste der Vergleiche: Satz 1 mit Satz 2, Satz 1 mit Satz 3, Satz 2 mit Satz 3
    comparisons = [("Satz 1 mit Satz 2", satz1, satz2), 
                   ("Satz 1 mit Satz 3", satz1, satz3),
                   ("Satz 2 mit Satz 3", satz2, satz3)]
    
    # Vergleiche jedes Satzpaar und berechne die Cosine Similarity
    for comparison_label, text1, text2 in comparisons:
        if not text1 or not text2:
            continue
        
        start_time = time.perf_counter()
        similarity = scaling_sentemb_similarity(text1, text2, tokenizer, model)
        end_time = time.perf_counter()
        
        elapsed_time_ms = round((end_time - start_time) * 1000, 9)
        
        # Ergebnisse speichern
        results.append({
            "Vergleich": comparison_label,
            "Cosine Similarity": similarity,
            "Berechnungszeit (ms)": elapsed_time_ms
        })

# Erstelle ein DataFrame aus den Ergebnissen
results_df = pd.DataFrame(results)

# Gib die Ergebnisse in einem formatierten DataFrame aus
print(results_df.to_string(index=False))

Lade das scaling_sentemb Modell...


C:\Users\Finn\Work\Bachelor\Bachelor\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Modell erfolgreich geladen.


ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.